In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from attack import TemplateAttack
from io_dat.traces_bin import TracesBin

In [ ]:
trace_folder = "</path/to/traces/folder/>"
template_folder = "</path/to/templates/folder/>"

Retrive all traces (and plaintext) files and check if there is a file for each key byte:

In [ ]:
all_trace_files = np.sort(
    [trace_folder + f for f in os.listdir(trace_folder)]).tolist()
assert len(all_trace_files) == 256

Select the proper _Template Attack_ class based on the analyzed cipher. It has a few optional parameters:
- `n_traces`: Determine how many traces use to profile for each key (default uses all traces in file).
- `mode`: Leakage point on where to perform the attack. Change based of the cipher.
- `target_byte`: Determine on which intermediate byte profile the Template (default is byte 0).
- `filter`: Apply a highpass filter to traces as preprocessing (default is False).
- `aggregate_n_samples`: Determine how many consecutive samples to average together as preprocessing (default is 1).
- `n_principal_comp`: Determine how many principal components use in PCA (default is 10).

In [ ]:
ta = TemplateAttack(template_folder)

## Profile Template

Compute the principal components and save them in `template_folder`:

In [ ]:
ta.computePCA(all_trace_files)
ta.fit(all_trace_files)

## Compute Guessing Metrics

In [ ]:
attack_folder = "</path/to/attack/traces/folder/>"
trace_files = np.sort(
    [attack_folder + f for f in os.listdir(attack_folder)]).tolist()

Compute, save and plot metrics. In folder `template_folder` there is a .txt summary of the principal side-channel metrics.

In [ ]:
ge, gd, ranks = ta.guessingMetrics(trace_files)
print(f"Guessing distance: {gd}, Guessing entropy: {ge}")

In [ ]:
fig = plt.figure(figsize=(8, 5))
plt.plot(np.mean(ranks, axis=0))
plt.grid(True)
plt.ylim(0, 180)

## Attack a single file

Load traces and plaintext to perform the attack:

In [ ]:
attack_folder = "</path/to/attack/traces/folder/>"

In [ ]:
listdir = np.sort(os.listdir(attack_folder)).tolist()
trace_file = listdir[0]

print(attack_folder+trace_file)

In [ ]:
trabin = TracesBin(attack_folder+trace_file)
atkTraces, atkPTexts = trabin.getAllTraces()
atkKey = trabin.getKey()[0]

Performed the actual attack and plot the result:

In [ ]:
p_k = ta.predict(atkTraces, atkPTexts)

In [ ]:
fsize = 20
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(111)
ax1.set_xlabel('N. traces', fontsize=fsize)
ax1.set_ylabel('Relative probability', fontsize=fsize)
data = np.cumsum(p_k, axis=0)

plt.rcParams.update({'font.size': fsize*0.8})
plt.grid(True)
for i in range(256):
    if i != atkKey:
        plt.plot(data[:, i]-np.min(data, 1),
                 linewidth=1.5, alpha=0.6, color='grey')
plt.plot(data[:, atkKey]-np.min(data, 1), linewidth=3,
         color='black', label="Attack key")
plt.legend(fontsize=fsize*0.9)